In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-retrievers-bm25

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM

# 加载本地模型
embed_model_path = "/home/work/var/data/ssr-share-data/m3e-base/"
llm_model_path = "/home/work/var/data/ssr-share-data/Qwen1.5-7B-Chat/"

# 使用与8.1相同的数据加载方式
documents = SimpleDirectoryReader('temp_docs').load_data()

# 初始化嵌入模型
embed_model = HuggingFaceEmbedding(model_name=embed_model_path)

# 创建向量索引
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# 创建基本向量检索引擎
vector_retriever = vector_index.as_retriever(similarity_top_k=2)

No sentence-transformers model found with name /home/work/var/data/ssr-share-data/m3e-base/. Creating a new one with MEAN pooling.


In [5]:

"""BM25的检索"""
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.node_parser import SentenceSplitter

# 创建BM25检索器
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)
bm25_retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=2)

In [6]:
from llama_index.core import PromptTemplate
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
import torch

# 初始化大语言模型
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == 'assistant':
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt

def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"


llm = HuggingFaceLLM(
    model_name=llm_model_path,
    tokenizer_name=llm_model_path,
    context_window=3900,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95, "pad_token_id": 151645},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
)

# 创建自动问题生成检索器
auto_merging_retriever = AutoMergingRetriever(
    vector_index.as_retriever(similarity_top_k=2),
    llm,
    verbose=True
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.87s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
import time
import torch
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# 加载本地模型
embed_model_path = "/home/work/var/data/ssr-share-data/m3e-base/"
llm_model_path = "/home/work/var/data/ssr-share-data/Qwen1.5-7B-Chat/"

# 设置全局嵌入模型
Settings.embed_model = HuggingFaceEmbedding(model_name=embed_model_path)

# 加载数据
documents = SimpleDirectoryReader('temp_docs').load_data()

# 初始化大语言模型
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == 'assistant':
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt

def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"


llm = HuggingFaceLLM(
    model_name=llm_model_path,
    tokenizer_name=llm_model_path,
    context_window=3900,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95, "pad_token_id": 151645},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
)

# 创建向量索引
vector_index = VectorStoreIndex.from_documents(documents)

# 创建检索器
vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=3)

# 创建BM25检索器
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)
bm25_retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=2)

# 测试问题
test_questions = [
    "哪些国家有著名的自然景观？",
    "有哪些建于公元前的历史遗迹？",
    "世界上最高的现代建筑是什么？",
    "哪些宗教场所最受游客欢迎？",
    "有哪些著名的桥梁地标？"
]

# 比较不同检索策略的性能
retrievers = {
    "Vector": vector_retriever,
    "BM25": bm25_retriever,
    "auto": auto_merging_retriever
}

results = {}

for name, retriever in retrievers.items():
    query_engine = RetrieverQueryEngine.from_args(
        retriever, 
        llm=llm,
        node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)]
    )
    start_time = time.time()
    responses = []
    for question in test_questions:
        response = query_engine.query(question)
        responses.append(str(response))
    end_time = time.time()
    results[name] = {
        "responses": responses,
        "time": end_time - start_time
    }

# 打印结果
for name, result in results.items():
    print(f"Strategy: {name}")
    print(f"Total time: {result['time']:.2f} seconds")
    for i, response in enumerate(result['responses']):
        print(f"Q{i+1}: {test_questions[i]}")
        print(f"A{i+1}: {response[:150]}...")  # 打印前150个字符
    print("-" * 50)

No sentence-transformers model found with name /home/work/var/data/ssr-share-data/m3e-base/. Creating a new one with MEAN pooling.
Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.17s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Strategy: Vector
Total time: 17.35 seconds
Q1: 哪些国家有著名的自然景观？
A1: Puerto Rico、Botswana和Tunisia都有著名的自然景观。Puerto Rico的自然景观位于Lake Scottfort，建于公元前403年，以其文化价值吸引游客；Botswana的自然景观位于Grenville，建于1289年，以其历史意义闻名；Tunisia的自然景观位于Sa...
Q2: 有哪些建于公元前的历史遗迹？
A2: 以下是根据提供的文件路径中的信息列出的建于公元前的历史遗迹：

1. Somalia的宏伟纪念碑：位于Schroederhaven，建于公元前-2394年
2. Suriname的美丽历史遗迹：位于Tonyfurt，建于公元前-1358年
3. Germany的令人印象深刻的历史遗迹：位于Port ...
Q3: 世界上最高的现代建筑是什么？
A3: 抱歉，根据您提供的文件内容，没有提及世界上最高的现代建筑。这些信息分别提到了French Southern Territories、United States of America和Uzbekistan的现代建筑，但它们的高度并没有提及。如果您需要关于世界最高现代建筑的信息，可能需要查阅最新的建筑记...
Q4: 哪些宗教场所最受游客欢迎？
A4: 根据提供的文件路径，以下是受游客欢迎的宗教场所：

1. **United States Virgin Islands** - Duaneside：这个独特的宗教场所因其历史价值和文化闻名，每年吸引来自全世界的游客。

2. **Kyrgyz Republic** - Mitchellfort：作为...
Q5: 有哪些著名的桥梁地标？
A5: 以下是三个著名的桥梁地标：

1. **Syrian Arab Republic的宏伟桥梁** - 位于North Christybury，建于公元141年，因其历史意义吸引游客。
2. **Wallis and Futuna的宏伟桥梁** - 位于North Ryantown，建于公元前-912年...
--------------------------------------------------
Strategy: BM25
Total time: